# EDA and Data Loading

This notebook demonstrates:
- Loading CPG sales data from parquet format
- Exploratory Data Analysis (EDA)
- Data quality checks
- Basic visualizations
- Data ingestion setup for Databricks (optional)


In [ ]:
# Import data loader and libraries
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

from src.data_loader import load_cpg_data, get_data_summary
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")


In [ ]:
# Load data (PySpark or pandas depending on environment)
data_path = '../data/cpg_sales_data.parquet'

try:
    df = load_cpg_data(data_path)
    print(f"✅ Data loaded successfully!")
    print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
except FileNotFoundError:
    print(f"❌ Data file not found at {data_path}")
    print("💡 Tip: Make sure the parquet file exists or generate sample data first")
    df = None


## Basic Data Exploration


In [ ]:
# Display first few rows
if df is not None:
    print("📋 First 5 rows:")
    display(df.head())
    
    print("\n📊 Column information:")
    print(df.info())
    
    print("\n📈 Basic statistics:")
    display(df.describe())
    
    print("\n📅 Date range:")
    if 'date' in df.columns:
        print(f"   From: {df['date'].min()}")
        print(f"   To: {df['date'].max()}")
        print(f"   Total days: {(df['date'].max() - df['date'].min()).days}")


In [ ]:
# Data quality checks
if df is not None:
    print("🔍 Data Quality Checks:")
    print("=" * 50)
    
    # Check for missing values
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print("\n⚠️  Missing values:")
        print(missing[missing > 0])
    else:
        print("\n✅ No missing values found!")
    
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"\n📋 Duplicate rows: {duplicates}")
    
    # Check data types
    print("\n📊 Data types:")
    print(df.dtypes)
    
    # Get summary statistics
    print("\n📈 Dataset Summary:")
    summary = get_data_summary(df)
    for key, value in summary.items():
        print(f"   {key}: {value}")


## Visualizations


In [ ]:
# Visualize distributions
if df is not None and 'revenue' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Revenue distribution
    axes[0, 0].hist(df['revenue'], bins=50, edgecolor='black')
    axes[0, 0].set_title('Revenue Distribution')
    axes[0, 0].set_xlabel('Revenue')
    axes[0, 0].set_ylabel('Frequency')
    
    # Revenue over time
    if 'date' in df.columns:
        daily_revenue = df.groupby('date')['revenue'].sum()
        axes[0, 1].plot(daily_revenue.index, daily_revenue.values)
        axes[0, 1].set_title('Revenue Over Time')
        axes[0, 1].set_xlabel('Date')
        axes[0, 1].set_ylabel('Total Revenue')
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Revenue by category
    if 'category' in df.columns:
        category_revenue = df.groupby('category')['revenue'].sum().sort_values(ascending=False)
        axes[1, 0].bar(range(len(category_revenue)), category_revenue.values)
        axes[1, 0].set_xticks(range(len(category_revenue)))
        axes[1, 0].set_xticklabels(category_revenue.index, rotation=45, ha='right')
        axes[1, 0].set_title('Revenue by Category')
        axes[1, 0].set_ylabel('Total Revenue')
    
    # Revenue by store region
    if 'store_region' in df.columns:
        region_revenue = df.groupby('store_region')['revenue'].sum()
        axes[1, 1].pie(region_revenue.values, labels=region_revenue.index, autopct='%1.1f%%')
        axes[1, 1].set_title('Revenue by Region')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizations created!")
